In [71]:
import numpy as np
import os
import torch
from misc.example_helper import *
import importlib
import processing
import models
%load_ext autoreload
%autoreload 2
importlib.reload(processing)
importlib.reload(models)
from processing.data_manager import SHREDDataManager
from processing.parametric_data_manager import ParametricSHREDDataManager
from models.shred_models import SHRED
from models.shred_models import evaluate
import os


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [129]:
# Initialize SHREDDataManager
manager = SHREDDataManager(
    lags = 20,
    train_size = 0.6,
    val_size = 0.2,
    test_size = 0.2,
    scaling=True,
    # scaling = False,
    compression=1000,
    time=np.arange(0, 2000),
    method="all"
)

input_folder = "data4fields_npz"
filename = 'ni_3D.npz'

file_path = os.path.join(input_folder, filename)
ni_3D = np.load(file_path)
ni_3D = ni_3D[ni_3D.files[0]]
mobile_sensors = [
    perimeter_walk(height = ni_3D.shape[1], width = ni_3D.shape[2], timesteps = ni_3D.shape[0], clockwise=True),
    perimeter_walk(height = ni_3D.shape[1], width = ni_3D.shape[2], timesteps = ni_3D.shape[0], clockwise=False)
]
manager.add_field(
    data=ni_3D,
    random_sensors=3,
    # stationary_sensors=[(7, 9), (0,0)],
    # mobile_sensors=mobile_sensors,
    id = str(os.path.splitext(filename)[0])
)

self.random_indices["train"] [1170  873  617 ...  578 1947 1797]
full_state_data_std_scaled [[ 4.30976052e+00  4.44095815e+00  4.38182420e+00 ...  4.45344336e+00
   3.85123729e+00  3.95069582e+00]
 [ 3.75471217e+00  3.45825688e+00  3.87210918e+00 ...  3.60383155e+00
   3.94922601e+00  3.77523108e+00]
 [ 3.29763163e+00  2.22609454e+00  1.99192786e+00 ...  2.73051545e+00
   2.42985383e+00  1.23016115e+00]
 ...
 [-2.68888512e-01 -6.65562929e-02  3.01027629e-01 ...  1.33975956e+00
   8.46093521e-01  1.33655837e+00]
 [-5.15176836e-04  1.46402166e-01  6.71817779e-01 ...  8.65049878e-01
   3.97095755e-01  6.56311357e-01]
 [ 2.06339915e-01  5.99417091e-01  5.29654568e-01 ...  7.54637771e-01
   5.93229481e-01  9.28349711e-01]]
self.full_state_data pre transform (2000, 65792)
transformed_data_std_scale (2000, 65792)
transformed_data (2000, 1000)
full_state_data_std_scaled [[ 4.04778707  4.15656717  4.07324084 ...  4.31877432  3.72084887
   3.87836434]
 [ 3.53314664  3.2490844   3.6050398  ...  3

In [130]:
manager.input_summary

,field id,sensor id,sensor type,location/trajectory
0,ni_3D,ni_3D-0,stationary (randomly selected),"(113, 66)"
1,ni_3D,ni_3D-1,stationary (randomly selected),"(18, 209)"
2,ni_3D,ni_3D-2,stationary (randomly selected),"(34, 183)"


In [131]:
manager.sensor_measurements # always shows unscaled, raw sensor data

,time,ni_3D-0,ni_3D-1,ni_3D-2
0,0,5.017400e+16,4.833600e+16,4.758400e+16
1,1,5.105000e+16,4.972500e+16,4.786500e+16
2,2,5.322400e+16,5.138600e+16,4.750200e+16
3,3,5.612900e+16,5.220400e+16,4.739200e+16
4,4,5.732000e+16,5.214700e+16,4.823800e+16
...,...,...,...,...
1995,1995,1.750600e+17,6.676100e+16,8.909800e+16
1996,1996,1.581400e+17,6.636200e+16,9.512100e+16
1997,1997,1.410100e+17,5.860100e+16,9.906000e+16
1998,1998,1.286100e+17,5.704300e+16,1.041700e+17


In [ ]:
train_set, val_set, test_set = manager.preprocess()

In [ ]:
ni_3D_train_DM = train_set.predictor_dataset.Y.numpy()
ni_3D_train_DM.shape

(1200, 1000)

In [134]:
ni_3D_train = ni_3D[0:1200,:]
ni_3D_train = ni_3D_train.reshape(ni_3D_train.shape[0], -1)
ni_3D_train.shape

(1200, 65792)

In [135]:
ni_3D_train_DM

array([[1.        , 0.32971737, 0.49816957, ..., 0.5959017 , 0.39603755,
        0.45326614],
       [0.99520165, 0.33104098, 0.49803093, ..., 0.50374556, 0.45187616,
        0.48809505],
       [0.9889164 , 0.33421493, 0.49737728, ..., 0.32912353, 0.56991506,
        0.56899697],
       ...,
       [0.01988333, 0.43880913, 0.4288136 , ..., 0.657122  , 0.348138  ,
        0.55649877],
       [0.02024428, 0.45084876, 0.3163485 , ..., 0.36948106, 0.5720663 ,
        0.41384125],
       [0.02063805, 0.46200365, 0.2547258 , ..., 0.52083355, 0.45105964,
        0.54179835]], shape=(1200, 1000), dtype=float32)

In [136]:
ni_3D_train

array([[5.6167e+16, 5.5958e+16, 5.5041e+16, ..., 5.6783e+16, 5.2811e+16,
        5.4436e+16],
       [5.2558e+16, 4.9793e+16, 5.1893e+16, ..., 5.1335e+16, 5.3438e+16,
        5.3275e+16],
       [4.9586e+16, 4.2063e+16, 4.0281e+16, ..., 4.5735e+16, 4.3716e+16,
        3.6435e+16],
       ...,
       [3.1246e+16, 3.3379e+16, 3.1204e+16, ..., 3.3453e+16, 3.4206e+16,
        4.0513e+16],
       [3.5052e+16, 2.9534e+16, 2.9089e+16, ..., 3.6916e+16, 3.9387e+16,
        3.8081e+16],
       [3.5691e+16, 3.3199e+16, 3.0193e+16, ..., 3.9615e+16, 3.9410e+16,
        3.4756e+16]], shape=(1200, 65792))

In [ ]:
unscaled = manager.postprocess(data = ni_3D_train_DM, uncompress = True, unscale = True, method = 'predictor')

method temporal_reconstructor
(1200, 1000)
unflattend


In [138]:
unscaled

{'ni_3D': array([[[5.59927565e+16, 5.34564525e+16, 5.34796037e+16, ...,
          4.63183236e+16, 5.39506443e+16, 5.13967644e+16],
         [5.39037926e+16, 4.74510005e+16, 4.88397146e+16, ...,
          4.67884634e+16, 5.18782235e+16, 5.77414788e+16],
         [4.61161420e+16, 4.33364055e+16, 4.86440904e+16, ...,
          4.69028210e+16, 4.63880198e+16, 4.82317363e+16],
         ...,
         [4.56427210e+16, 5.06695633e+16, 4.70886834e+16, ...,
          4.64310046e+16, 5.06681616e+16, 5.48173315e+16],
         [4.82035902e+16, 5.39040091e+16, 4.50532430e+16, ...,
          5.02794265e+16, 4.92698895e+16, 5.35990303e+16],
         [6.08653677e+16, 5.81077356e+16, 4.60167379e+16, ...,
          5.53086565e+16, 5.33423474e+16, 5.26248297e+16]],
 
        [[5.40104522e+16, 5.13057463e+16, 5.15229995e+16, ...,
          4.53017242e+16, 5.21324515e+16, 5.10329637e+16],
         [5.18070463e+16, 4.62382883e+16, 4.76753277e+16, ...,
          4.58735955e+16, 5.01009877e+16, 5.57260216e+16]

In [ ]:
ni_3D_train_DM = train_set.reconstructor_dataset.Y.numpy()
ni_3D_train_DM.shape

(1200, 1000)

In [140]:
# 956 1834 1044
# 431  919 1611
# 1170  873  617
ni_3D_train = ni_3D[(1170, 873, 617),:]
ni_3D_train = ni_3D_train.reshape(ni_3D_train.shape[0], -1)
ni_3D_train.shape

(3, 65792)

In [141]:
ni_3D_train

array([[2.7722e+16, 2.4955e+16, 2.8830e+16, ..., 2.7285e+16, 3.1451e+16,
        2.9047e+16],
       [2.7264e+16, 2.5884e+16, 2.5925e+16, ..., 3.2112e+16, 3.1607e+16,
        2.5713e+16],
       [2.5821e+16, 2.2642e+16, 2.2391e+16, ..., 2.8047e+16, 2.7650e+16,
        3.1628e+16]], shape=(3, 65792))

In [ ]:
unscaled = manager.postprocess(data = ni_3D_train_DM, uncompress = True, unscale = True, method = 'reconstructor')

method random_reconstructor
(1200, 1000)
unflattend


In [143]:
unscaled

{'ni_3D': array([[[2.77463603e+16, 2.54470287e+16, 2.85210738e+16, ...,
          2.39559350e+16, 2.61683955e+16, 2.61865888e+16],
         [2.96648867e+16, 2.69116443e+16, 2.96741275e+16, ...,
          2.48813065e+16, 2.66400835e+16, 2.70162214e+16],
         [3.18599558e+16, 2.89159637e+16, 3.16433199e+16, ...,
          2.76573642e+16, 2.90974320e+16, 3.05756841e+16],
         ...,
         [3.76768219e+16, 3.86570955e+16, 3.93153323e+16, ...,
          3.31221176e+16, 3.71268795e+16, 3.71201734e+16],
         [3.63483074e+16, 3.81060921e+16, 3.59008234e+16, ...,
          2.86096708e+16, 3.36892581e+16, 3.27650893e+16],
         [3.59235186e+16, 3.77807775e+16, 3.40745015e+16, ...,
          2.75989579e+16, 3.23954116e+16, 3.03344493e+16]],
 
        [[2.73758094e+16, 2.54272491e+16, 2.56967842e+16, ...,
          2.46809761e+16, 2.35403240e+16, 2.55230528e+16],
         [2.86964225e+16, 2.56332933e+16, 2.65666478e+16, ...,
          2.65718922e+16, 2.44597338e+16, 2.55692587e+16]

In [ ]:
# initialize SHRED
shred = SHRED(sequence='LSTM', decoder='SDN')
# fit SHRED
val_errors = shred.fit(train_set, val_set, num_epochs=50, patience=20, verbose=True)

Newest Version
input_size 3
output_size 65792
self._sequence_model_random_reconstructor.output_size 64

Fitting Random Reconstructor...










































Epoch 1/50: 100%|██████████| 19/19 [00:03<00:00,  5.68batch/s, loss=0.12, L2=0.619, val_loss=0.0295, val_L2=0.334]








































Epoch 2/50: 100%|██████████| 19/19 [00:03<00:00,  5.76batch/s, loss=0.0296, L2=0.332, val_loss=0.0259, val_L2=0.312]





KeyboardInterrupt: 